In [1]:
# --- 1. Setup and Configuration ---

import os
import gc
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from scipy.ndimage import mean as ndimage_mean

class CFG:
    # General
    DEVICE = torch.device('cpu') # Forcing CPU usage to bypass persistent CUDA OOM errors
    
    # Data Paths
    VALID_PATH = 'train'
    CALIBRATION_FRAGMENT_ID = '2'

    # Data Reading (must match training)
    Z_START = 20
    Z_END = 44
    IN_CHANS = (Z_END - Z_START) + 1

    # Tiling (must match training)
    TILE_SIZE = 256
    STRIDE = TILE_SIZE // 2

    # Model
    BACKBONE = 'timm-efficientnet-b4'
    MODEL_PATH = 'best_finetuned_model.pth' # Evaluate the fine-tuned model

    # Inference Strategy
    USE_TTA = False
    BATCH_SIZE = 1

print(f"Device: {CFG.DEVICE}")
print(f"Evaluating Model: {CFG.MODEL_PATH}")
print(f"Calibration fragment: {CFG.CALIBRATION_FRAGMENT_ID}")

Device: cpu
Evaluating Model: best_finetuned_model.pth
Calibration fragment: 2


In [2]:
# --- 2. Helper Functions & Dataset ---

def get_hann_window(size):
    """Creates a 2D Hanning window."""
    hann_1d = np.hanning(size)
    hann_2d = np.outer(hann_1d, hann_1d)
    return hann_2d

def remove_small_components(mask, min_size):
    """Removes small connected components from a binary mask."""
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] < min_size:
            mask[labels == i] = 0
    return mask

def get_img_stack(fragment_id, z_start, z_end, data_path, simulate_ir_absence=False):
    """Loads and normalizes the image stack for a fragment."""
    images = []
    
    # Load TIF slices
    for i in range(z_start, z_end):
        image_path = os.path.join(data_path, fragment_id, 'surface_volume', f'{i:02}.tif')
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if image is None: raise FileNotFoundError(f"TIF file not found: {image_path}")
        images.append(image.astype(np.float32))

    # Load IR image or simulate its absence
    ir_path = os.path.join(data_path, fragment_id, 'ir.png')
    ir_image = cv2.imread(ir_path, cv2.IMREAD_UNCHANGED)
    
    if ir_image is None or simulate_ir_absence:
        if ir_image is None: print(f"Warning: IR file not found at '{ir_path}'.")
        print("IR Fallback: Using mean of TIF slices as IR channel.")
        ir_image = np.mean(np.stack(images, axis=0), axis=0).astype(np.float32)
    else:
        ir_image = ir_image.astype(np.float32)

    images.append(ir_image)
    
    # Per-channel percentile normalization
    normalized_images = []
    for img in images:
        p1, p99 = np.percentile(img, [1, 99])
        img_normalized = (img - p1) / (p99 - p1 + 1e-6)
        img_normalized = np.clip(img_normalized, 0, 1)
        normalized_images.append(img_normalized)
        
    return np.stack(normalized_images, axis=-1)

class VesuviusTestDataset(Dataset):
    """Dataset for inference on pre-processed images."""
    def __init__(self, tiles, fragment_images, tile_size):
        self.tiles = tiles
        self.fragment_images = fragment_images
        self.tile_size = tile_size

    def __len__(self):
        return len(self.tiles)

    def __getitem__(self, idx):
        y, x = self.tiles[idx]
        image_tile = self.fragment_images[y:y + self.tile_size, x:x + self.tile_size, :]
        image = np.transpose(image_tile, (2, 0, 1))
        return torch.from_numpy(image).float()

In [3]:
# --- 3. Model Loading, Prediction & Evaluation ---

# Define the model architecture (must match training)
model = smp.FPN(
    encoder_name=CFG.BACKBONE,
    encoder_weights=None,  # Weights will be loaded from file
    in_channels=CFG.IN_CHANS,
    classes=1,
    activation=None,
)

# Load the trained weights (CPU first, then move to GPU)
print(f"Loading model from {CFG.MODEL_PATH}...")
state_dict = torch.load(CFG.MODEL_PATH, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.to(CFG.DEVICE)
model.eval()
print("Model loaded successfully.")

def tta_predict(model, image_batch):
    """Performs 8-way Test-Time Augmentation and returns averaged logits."""
    B, C, H, W = image_batch.shape
    logits_tta = torch.zeros((B, 1, H, W), device=image_batch.device, dtype=image_batch.dtype)

    # Original
    logits_tta += model(image_batch)

    # Horizontal Flip
    logits_tta += torch.flip(model(torch.flip(image_batch, dims=[3])), dims=[3])

    # Rotations (90, 180, 270) and their flips
    for k in [1, 2, 3]:
        img_rot = torch.rot90(image_batch, k, [2, 3])
        logits_tta += torch.rot90(model(img_rot), -k, [2, 3])
        logits_tta += torch.flip(torch.rot90(model(torch.flip(img_rot, dims=[3])), -k, [2, 3]), dims=[3])

    return logits_tta / 8.0

def predict_fragment(model, fragment_images, roi_mask):
    """Runs full-image inference on a fragment using overlapping tiles and Hanning window blending."""
    img_height, img_width, _ = fragment_images.shape
    logit_canvas = np.zeros((img_height, img_width), dtype=np.float32)
    weight_canvas = np.zeros((img_height, img_width), dtype=np.float32)
    hann_window = get_hann_window(CFG.TILE_SIZE)
    
    tiles = []
    for y in range(0, img_height, CFG.STRIDE):
        for x in range(0, img_width, CFG.STRIDE):
            y_start = min(y, img_height - CFG.TILE_SIZE)
            x_start = min(x, img_width - CFG.TILE_SIZE)
            if (roi_mask[y_start:y_start+CFG.TILE_SIZE, x_start:x_start+CFG.TILE_SIZE] > 0).mean() > 0.1:
                if (y_start, x_start) not in tiles:
                    tiles.append((y_start, x_start))
    
    print(f"Generated {len(tiles)} tiles for prediction.")
    dataset = VesuviusTestDataset(tiles, fragment_images, CFG.TILE_SIZE)
    dataloader = DataLoader(dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    
    with torch.no_grad():
        for i, images_batch in enumerate(tqdm(dataloader, desc="Predicting tiles")):
            images_batch = images_batch.to(CFG.DEVICE)
            
            if CFG.USE_TTA:
                logits_batch = tta_predict(model, images_batch)
            else:
                logits_batch = model(images_batch)
            
            logits_batch = logits_batch.cpu().numpy()
            
            for j, (y, x) in enumerate(tiles[i*CFG.BATCH_SIZE : (i+1)*CFG.BATCH_SIZE]):
                logit_canvas[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE] += logits_batch[j, 0, :, :] * hann_window
                weight_canvas[y:y+CFG.TILE_SIZE, x:x+CFG.TILE_SIZE] += hann_window
    
    logit_canvas /= (weight_canvas + 1e-6)
    return logit_canvas

def fbeta_score(y_true, y_pred, beta=0.5):
    """Calculates the F-beta score."""
    tp = np.sum(y_true * y_pred)
    fp = np.sum((1 - y_true) * y_pred)
    fn = np.sum(y_true * (1 - y_pred))
    
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + 1e-6)
    return fbeta, precision, recall

def evaluate_model(model):
    """
    Evaluates the model on the calibration fragment by simulating test conditions (missing IR).
    Performs a grid search to find the best threshold and min_area_size.
    """
    print("\n--- Starting Model Evaluation ---")
    fragment_id = CFG.CALIBRATION_FRAGMENT_ID
    
    # Load validation data
    print(f"Loading validation fragment {fragment_id} for evaluation...")
    roi_mask = cv2.imread(os.path.join(CFG.VALID_PATH, fragment_id, 'mask.png'), cv2.IMREAD_GRAYSCALE)
    gt_mask = cv2.imread(os.path.join(CFG.VALID_PATH, fragment_id, 'inklabels.png'), cv2.IMREAD_GRAYSCALE)
    gt_mask = (gt_mask > 0).astype(np.uint8)
    
    # Load images and get full-fragment logit predictions
    fragment_images = get_img_stack(fragment_id, CFG.Z_START, CFG.Z_END, data_path=CFG.VALID_PATH, simulate_ir_absence=True)
    logit_map = predict_fragment(model, fragment_images, roi_mask)
    prob_map = 1 / (1 + np.exp(-logit_map))
    
    # Optimized Grid Search
    print("Performing grid search for best parameters...")
    thresholds = np.arange(0.20, 0.80, 0.025)
    min_areas = [64, 96, 128, 160, 196, 256, 300]
    best_score = -1
    best_params = (0.0, 0)

    pred_mask_base = (prob_map > thresholds.min()).astype(np.uint8)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(pred_mask_base, connectivity=8)
    component_areas = stats[1:, cv2.CC_STAT_AREA]
    component_probs = ndimage_mean(prob_map, labels=labels, index=np.arange(1, num_labels))
    gt_pixels_roi = gt_mask[roi_mask > 0]

    for threshold in tqdm(thresholds, desc="Thresholds"):
        for min_area in min_areas:
            passing_indices = np.where((component_probs > threshold) & (component_areas >= min_area))[0] + 1
            pred_mask = np.isin(labels, passing_indices).astype(np.uint8)
            pred_pixels_roi = pred_mask[roi_mask > 0]
            score, _, _ = fbeta_score(gt_pixels_roi, pred_pixels_roi)
            
            if score > best_score:
                best_score = score
                best_params = (threshold, min_area)

    print(f"\n--- Evaluation Complete ---")
    print(f"Best F0.5 score: {best_score:.4f}")
    print(f"Best parameters found: Threshold={best_params[0]:.2f}, Min Area={best_params[1]}")
    
    return best_score, best_params

# Run the evaluation
evaluate_model(model)

Loading model from best_finetuned_model.pth...
Model loaded successfully.

--- Starting Model Evaluation ---
Loading validation fragment 2 for evaluation...


IR Fallback: Using mean of TIF slices as IR channel.


Generated 6253 tiles for prediction.


Predicting tiles:   0%|          | 0/6253 [00:00<?, ?it/s]

Predicting tiles:   0%|          | 1/6253 [00:00<24:45,  4.21it/s]

...

Thresholds:  96%|█████████▌| 24/25 [01:07<00:01,  1.84s/it]

Thresholds: 100%|██████████| 25/25 [01:08<00:00,  1.73s/it]

Thresholds: 100%|██████████| 25/25 [01:08<00:00,  2.74s/it]


--- Evaluation Complete ---
Best F0.5 score: 0.1850
Best parameters found: Threshold=0.20, Min Area=64


(0.18502275050104208, (0.2, 64))